<a href="https://colab.research.google.com/github/vikas123456778009/PubMed/blob/main/pubmeds_project_Vikas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install biopython --upgrade
%pip install requests_html
%pip install pdfplumber


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 131.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 49.0 MB/s eta 0:00:00


In [3]:
from Bio import Entrez
from requests_html import HTMLSession
import requests
from requests.exceptions import ConnectionError


def search_pmc(search_term, max_results=10):
    Entrez.email = "savicisplaying@gmail.com"  # Set your email for NCBI API access

    # Use the esearch function to search for articles in PMC
    handle = Entrez.esearch(db="pmc", term=search_term, retmax=max_results)
    record = Entrez.read(handle)
    handle.close()

    return record["IdList"]

def main():
    search_term = input("Enter the search term for PMC: ")
    max_results = int(input("Enter the maximum number of results to fetch: "))

    pmc_ids = search_pmc(search_term, max_results)

    s = HTMLSession()

    for pmc in pmc_ids:
      try:
        pmcid = pmc.strip()
        base_url = 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC'
        r = s.get(base_url + pmcid + '/', timeout=3)
        pdf_url = 'https://www.ncbi.nlm.nih.gov/' + r.html.find('a.int-view', first=True).attrs['href']
        r=s.get(pdf_url, stream=True)
        with open(pmcid + '.pdf', 'wb') as f:
          for chunk in r.iter_content(chunk_size=1024):
            if chunk:
              f.write(chunk)


      except ConnectionError as e:
        pass


if __name__ == "__main__":
    main()


Enter the search term for PMC: Monkey Pox
Enter the maximum number of results to fetch: 2


In [ ]:
import pdfplumber
from transformers import BartForConditionalGeneration, BartTokenizer
import re
import os

# Step 1: Convert PDF to Text
def pdf_to_text(pdf_file):
    with pdfplumber.open(pdf_file) as pdf:
        text = ""
        for page in pdf.pages:
            # Extract text from the PDF page
            page_text = page.extract_text()

            # Remove image tags from the text
            page_text = re.sub(r'\<\/?[img|IMG|Image]\>|\[[A-Za-z]+\]', '', page_text)

            text += page_text
    return text

# Step 2: Preprocess the Text Data
def preprocess(text):
    # Remove unnecessary characters and formatting
    text = re.sub(r'[\n\r\t]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text

# Step 3: Load the Pretrained Summarization Model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Step 4: Summarize the Text Data
def summarize(text, min_words=100, max_words=150):
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs["input_ids"], num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # Adjust the summary length
    summary_words = summary.split()
    if len(summary_words) < min_words:
        return summary
    elif len(summary_words) > max_words:
        return ' '.join(summary_words[:max_words])
    else:
        return summary

# Usage
pdf_dir = "/content"
summary_dir = "/content/summary"
combined_summary_file = "combined_summaries.txt"

# Iterate over PDF files and generate summaries
combined_summaries = []
for pdf_file in os.listdir(pdf_dir):
    if pdf_file.endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, pdf_file)
        text = pdf_to_text(pdf_path)
        preprocessed_text = preprocess(text)
        summary = summarize(preprocessed_text, min_words=100, max_words=150)

        # Get the PubMed ID from the file name
        pubmed_id = os.path.splitext(pdf_file)[0]

        # Add the summary to the combined list with its PubMed ID as the heading
        combined_summaries.append(f"PubMed ID: {pubmed_id}\n\n{summary}\n\n")

# Save the combined summaries to a text file
with open(combined_summary_file, "w", encoding="utf-8") as f:
    f.write("\n".join(combined_summaries))

print(f"Combined summaries saved: {combined_summary_file}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Combined summaries saved: combined_summaries.txt


In [ ]:
import pandas as pd
pf = pd.read_csv("/content/combined_summaries.txt", sep=r"\n",lineterminator='-' ,header=None)
pf.head()

<ipython-input-13-e5b58b72c3cf>:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  pf = pd.read_csv("/content/combined_summaries.txt", sep=r"\n",lineterminator='-' ,header=None)


,0
0,PubMed ID: 11335426
1,A male patient in his 40s was diagnosed with r...
2,PubMed ID: 11335428
3,Lower extremity function and cardiovascular di...


from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby(0).size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)